In [ ]:
%load_ext autoreload
%autoreload 2

In [194]:
import sepsis_labeler as sl
from sepsis_labeler.cohort import SepsisAdmissionCohort
from sepsis_labeler.component_base import Component
from sepsis_labeler.starr_flowsheet_extract import STARRFlowsheetExtract
import sepsis_labeler.component as slc
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
config_dict = {
    "dataset_project": "som-nero-nigam-starr",
    "rs_dataset_project": "som-nero-nigam-starr",
    "dataset": "cdm_subset",
#     "dataset": "starr_omop_cdm5_deid_2022_08_01",
    "rs_dataset": "jlemmon_explore",
    "ext_flwsht_table":"meas_vals_json",
#     "print_query": True,
}
cohort_names = {
    "admission_rollup": "sepsis_1p_cohort",
    "suspected_infection": "sepsis_susp_inf_rollup",
    "sepsis_platelet": "sepsis_platelet_rollup",
    "sepsis_creatinine": "sepsis_creatinine_rollup",
    "sepsis_bilirubin": "sepsis_bilirubin_rollup",
    "sepsis_lactate": "sepsis_lactate_rollup",
    "sepsis_urine": "sepsis_urine_rollup",
    "sepsis_pao2_fio2": "sepsis_pao2_fio2_rollup",
    "sepsis_spo2_fio2": "sepsis_spo2_fio2_rollup",
    "sepsis_map": "sepsis_map_rollup",
    "sepsis_gcs": "sepsis_gcs_rollup",
    "sepsis_vent": "sepsis_vent_rollup",
    "sepsis_vasopressor": "sepsis_vassopressor_rollup",
}
cohort_names_long = {
    key: "{rs_dataset_project}.{rs_dataset}.{cohort_name}".format(
        cohort_name=value, **config_dict
    )
    for key, value in cohort_names.items()
}
config_dict = {**config_dict, **cohort_names_long}

In [ ]:
sfe = STARRFlowsheetExtract(**config_dict)

In [134]:
sc = SepsisCohort(**config_dict)
sc.create_cohort_table()

In [135]:
cohort_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{admission_rollup}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
cohort_df

,person_id,admit_date,discharge_date
0,29927014,2011-05-13 17:08:00,2011-05-14 15:13:00
1,29927175,2011-01-21 11:45:00,2011-01-22 16:15:00
2,29927202,2014-06-26 17:36:00,2014-07-07 13:59:00
3,29927202,2014-07-09 14:35:00,2014-07-11 17:58:00
4,29927202,2014-07-20 18:25:00,2014-07-30 16:06:00
...,...,...,...
2918,72381963,2021-09-04 05:22:00,2021-09-13 14:11:00
2919,72404313,2021-11-24 17:05:00,2021-11-27 13:40:00
2920,72434661,2021-11-27 18:36:00,2021-12-10 14:28:00
2921,72988345,2021-11-02 18:30:00,2021-11-06 06:19:00


In [136]:
si = slc.SuspectedInfectionComponent(**config_dict)
si.create_component_table()

In [137]:
sus_inf_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{suspected_infection}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
sus_inf_df

,person_id,admit_date,admit_datetime,discharge_date,discharge_datetime,min_bc,min_systemic_abx,index_date
0,30068253,2016-11-12,2016-11-12 23:30:00,2016-11-16,2016-11-16 18:56:00,2016-11-12 17:55:00,2016-11-12,2016-11-12
1,30188667,2019-07-23,2019-07-23 19:28:00,2019-07-26,2019-07-26 13:25:00,2019-07-23 22:36:00,2019-07-23,2019-07-23
2,30188667,2019-10-17,2019-10-17 18:43:00,2019-10-22,2019-10-22 11:23:00,2019-10-17 19:05:00,2019-10-17,2019-10-17
3,30313608,2010-09-23,2010-09-23 22:13:00,2010-09-26,2010-09-26 21:46:00,2010-09-23 23:03:00,2010-09-23,2010-09-23
4,30370183,2020-06-15,2020-06-15 14:12:00,2020-06-19,2020-06-19 12:01:00,2020-06-15 15:08:00,2020-06-15,2020-06-15
...,...,...,...,...,...,...,...,...
863,31538664,2015-05-04,2015-05-04 09:47:00,2015-05-10,2015-05-10 16:31:00,2015-05-04 09:53:00,2015-05-04,2015-05-04
864,31811595,2014-10-20,2014-10-20 14:06:00,2014-10-23,2014-10-23 17:36:00,2014-10-20 21:46:00,2014-10-20,2014-10-20
865,32058298,2017-08-07,2017-08-07 05:10:00,2017-08-12,2017-08-12 15:20:00,2017-08-09 07:08:00,2017-08-08,2017-08-08
866,32061266,2016-10-03,2016-10-03 16:48:00,2016-10-18,2016-10-18 21:22:00,2016-10-03 16:11:00,2016-10-03,2016-10-03


In [159]:
pc = slc.PlateletComponent(**config_dict)
pc.create_component_table()

In [160]:
plate_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_platelet}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
plate_df

,person_id,admit_date,min_platelet
0,30732251,2020-05-24,2.0
1,31052356,2019-03-08,2.0
2,30929785,2021-04-28,5.0
3,30598763,2018-10-05,5.0
4,30383055,2021-03-26,6.0
...,...,...,...
859,30913480,2021-01-30,383.0
860,31140716,2008-12-27,447.0
861,31672873,2019-01-05,495.0
862,30453839,2017-03-18,495.0


In [161]:
pc = slc.PlateletComponent(prior=True,**config_dict)
pc.create_component_table()

In [162]:
plate_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_platelet}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
plate_df

,person_id,admit_date,min_platelet
0,31679746,2020-10-26,95.0
1,31679746,2020-11-21,58.0
2,30149635,2018-08-27,89.0
3,31677187,2010-11-07,195.0
4,44488453,2020-05-15,70.0
...,...,...,...
330,32064254,2010-04-11,18.0
331,32064254,2010-07-28,36.0
332,32064254,2010-12-04,283.0
333,32064254,2010-05-23,15.0


In [163]:
cc = slc.CreatinineComponent(**config_dict)
cc.create_component_table()

In [164]:
creat_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_creatinine}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
creat_df

,person_id,admit_date,max_creatinine
0,30330916,2009-02-19,1078.481639
1,29940147,2014-06-06,1078.481639
2,32247281,2020-05-09,581.672884
3,32061266,2016-10-03,1191.633811
4,30278209,2012-02-15,609.960927
...,...,...,...
859,32064254,2010-04-11,53.040081
860,32064254,2010-02-19,53.040081
861,32064254,2010-04-25,53.040081
862,32064254,2010-10-15,114.920175


In [165]:
cc = slc.CreatinineComponent(prior=True, **config_dict)
cc.create_component_table()

In [166]:
creat_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_creatinine}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
creat_df

,person_id,admit_date,max_creatinine
0,30012432,2020-10-07,539.240820
1,32060915,2015-01-29,574.600873
2,30716432,2020-04-02,837.149272
3,30012432,2020-10-20,872.509326
4,30457626,2019-06-07,301.444458
...,...,...,...
321,30404957,2019-06-16,447.304680
322,30716432,2020-01-09,1306.553986
323,30071508,2019-08-24,230.724351
324,30049211,2020-01-18,230.724351


In [167]:
gc = slc.GlasgowComaScaleComponent(**config_dict)
gc.create_component_table()

In [168]:
gcs_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_gcs}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
gcs_df

,person_id,admit_date,min_gcs
0,31115013,2015-04-24,3.0
1,30716432,2020-10-06,3.0
2,30716432,2020-04-02,3.0
3,31253273,2021-01-25,3.0
4,43747624,2019-12-05,3.0
...,...,...,...
487,31185911,2017-07-31,15.0
488,31525113,2019-01-12,15.0
489,30588409,2019-12-14,15.0
490,30588409,2019-05-09,15.0


In [169]:
gc = slc.GlasgowComaScaleComponent(prior=True, **config_dict)
gc.create_component_table()

In [170]:
gcs_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_gcs}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
gcs_df

,person_id,admit_date,min_gcs
0,30351808,2019-08-23,3.0
1,32563728,2021-03-30,6.0
2,30909752,2015-03-19,7.0
3,30923472,2013-03-07,8.0
4,30455795,2017-11-16,8.0
...,...,...,...
111,32516080,2017-06-12,15.0
112,32060915,2015-09-19,15.0
113,32060915,2015-01-18,15.0
114,32209139,2018-11-30,15.0


In [171]:
bc = slc.BilirubinComponent(**config_dict)
bc.create_component_table()

In [172]:
bili_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_bilirubin}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
bili_df

,person_id,admit_date,max_bilirubin
0,30505293,2019-12-18,2.0
1,30322757,2014-06-06,2.5
2,30044920,2008-07-26,2.5
3,32116582,2020-05-26,3.0
4,31028574,2015-02-12,3.0
...,...,...,...
805,32064254,2010-04-11,1.5
806,32064254,2010-02-19,0.6
807,32064254,2010-04-25,0.7
808,32064254,2010-05-23,2.1


In [173]:
bc = slc.BilirubinComponent(prior=True,**config_dict)
bc.create_component_table()

In [174]:
bili_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_bilirubin}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
bili_df

,person_id,admit_date,max_bilirubin
0,31522337,2021-12-10,4.0
1,32173593,2009-01-10,5.0
2,31007877,2011-07-09,0.5
3,31768409,2016-02-14,0.5
4,31659136,2008-12-10,0.5
...,...,...,...
265,31253273,2020-01-11,0.9
266,30588587,2014-10-28,1.3
267,29974059,2011-08-03,1.8
268,31583751,2012-05-12,1.8


In [176]:
mvc = slc.MechanicalVentilationComponent(**config_dict)
mvc.create_component_table()

In [177]:
mv_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_vent}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
mv_df

,person_id,admit_date,count_vent_mode
0,31141415,2019-07-11,1
1,30859528,2015-02-21,1
2,31947544,2017-11-06,1
3,31142331,2016-10-03,1
4,30071508,2021-07-12,1
...,...,...,...
123,32485517,2016-05-29,212
124,30854260,2016-04-26,221
125,31404458,2016-12-31,230
126,30723917,2020-11-08,244


In [178]:
mvc = slc.MechanicalVentilationComponent(prior=True,**config_dict)
mvc.create_component_table()

In [179]:
mv_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_vent}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
mv_df

,person_id,admit_date,count_vent_mode
0,30859528,2015-02-21,1
1,32349166,2016-11-28,2
2,32597724,2020-05-08,2
3,30919117,2013-01-15,4
4,30048381,2019-04-08,4
5,32462255,2019-06-03,260
6,30909752,2015-03-19,5
7,32064254,2010-10-15,11
8,31583751,2012-05-12,12
9,30351808,2019-08-23,15


In [180]:
lc = slc.LactateComponent(**config_dict)
lc.create_component_table()

In [181]:
lac_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_lactate}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
lac_df

,person_id,admit_date,max_lactate
0,30920914,2020-04-07,2.00
1,32247281,2020-05-09,2.00
2,32214209,2017-11-03,2.00
3,32261567,2021-01-30,2.00
4,32247281,2020-05-07,2.00
...,...,...,...
611,31185911,2017-07-31,0.95
612,31185911,2015-01-16,1.10
613,31393533,2021-08-03,2.26
614,31393533,2021-03-19,2.60


In [182]:
lc = slc.LactateComponent(prior=True,**config_dict)
lc.create_component_table()

In [183]:
lac_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_lactate}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
lac_df

,person_id,admit_date,max_lactate
0,32514739,2021-08-28,2.00
1,31923937,2019-11-27,2.00
2,32251166,2008-09-26,2.00
3,30912419,2018-09-07,5.50
4,30716432,2020-04-02,0.50
...,...,...,...
65,30487521,2011-04-06,1.24
66,32516080,2017-05-05,1.93
67,30321045,2017-01-25,0.81
68,30588233,2013-02-26,1.06


In [184]:
pfc = slc.PaO2FiO2Component(**config_dict)
pfc.create_component_table()

In [185]:
pf_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_pao2_fio2}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
pf_df

,person_id,admit_date,min_paO2fiO2_ratio
0,31115013,2015-04-24,252.500000
1,31583751,2012-05-12,180.000000
2,30716432,2020-10-06,54.000000
3,31253273,2021-01-25,86.000000
4,43747624,2019-12-05,121.666667
5,31009067,2015-10-11,202.500000
6,31180844,2015-12-11,196.000000
7,30873650,2014-07-04,97.142857
8,31864114,2010-09-23,54.000000
9,31141698,2010-03-21,101.250000


In [186]:
pfc = slc.PaO2FiO2Component(prior=True,**config_dict)
pfc.create_component_table()

In [187]:
pf_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_pao2_fio2}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
pf_df

,person_id,admit_date,min_paO2fiO2_ratio
0,31007877,2011-07-09,247.666667
1,30923472,2013-03-07,272.500000
2,31583751,2012-05-12,130.000000
3,30920982,2014-11-06,121.666667
4,30012432,2020-10-07,280.000000
5,32064254,2010-10-15,350.000000


In [188]:
sfc = slc.SpO2FiO2Component(**config_dict)
sfc.create_component_table()

In [189]:
pf_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_spo2_fio2}`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
pf_df

,person_id,admit_date,min_spO2fiO2_ratio


In [191]:
sfc = slc.SpO2FiO2Component(prior=True,**config_dict)
sfc.create_component_table()

In [192]:
pf_df = pd.read_gbq(
		"""
			SELECT *
			FROM `{sepsis_spo2_fio2}_prior`
		""".format(
			**config_dict
		),
		dialect='standard',
	)
pf_df

,person_id,admit_date,min_spO2fiO2_ratio
